In [1]:
from controller import DockerController
docker_controller = DockerController()

In [9]:
import random
import docker 
client = docker.from_env()
containers = client.containers.list(all=True)
for c in containers:
    if c.image.tags[0] == 'tf2:0.05':
        print(c.name)
        c.remove()

container_27
container_26
container_25
container_24
container_23
container_22
container_21
container_20
container_19
container_18
container_17
container_16
container_15
container_14
container_13
container_12
container_11
container_10
container_9
container_8
container_7
container_6
container_5
container_4
container_3
container_2
container_1
container_0


In [5]:
docker_controller.load_one("6deec6c04a49")

In [6]:
docker_controller.start_one(name="test", gpu=0, device=0, port=8080, user_dir="test")

Run: bbfb5f8b8224 gpu: 0 device: 0


In [13]:
import docker
docker_client = docker.from_env()
docker_client.containers.list()

[<Container: 655061174f1c>,
 <Container: 8ba386dea2ae>,
 <Container: 72e69a189526>,
 <Container: 899576c86f6a>,
 <Container: 7e443f11a0a3>,
 <Container: 6deec6c04a49>]

In [156]:
import docker
from nvitop import MigDevice
client = docker.from_env()
mig = MigDevice.all()

def get_gpu_status():
     gpu_status = []
     for gpu in mig:
          #free = gpu.memory_free()
          used = gpu.memory_used()
          gpu, device = gpu.index
          gpuStr = str(gpu)+":"+str(device)
          gpu_status.append(dict(gpu=gpuStr, used=used))
     return gpu_status
## python filter type to dict

def get_containers():
    containers = []
    gpu_status = get_gpu_status()
    for index, container in enumerate(client.containers.list(all=True)):
          if container.image.tags[0] in ["tf2:0.05", "tf1:0.01"]:
               cnt = dict(index=index, id=container.short_id, name=container.name, status=container.status)
               #gpu = list(filter(lambda x: x["index"] == index, gpu_status))
               my_dict = gpu_status[index]#gpu[0]
               containers.append({**cnt, **my_dict})
    return containers  

In [2]:
from influxdb_client import  Point, InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
from nvitop import MigDevice
import time
import datetime

SLEEP = 5
BUCKET = "gpu"
ORGANIZATION = "my-org"
TOKEN = "BwsRD6-_mbeV8IQUJjcAKJuj2pZir0pp9Cy1bezT3z0MJ5fkqD5wmhY_l5cnCbYLlgxsy0L8GkuJ0PM_n-sk6Q=="
URL = "http://localhost:8076"

client = InfluxDBClient(url=URL, token=TOKEN, org=ORGANIZATION)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()
try:
    while True:
        points = []
        mig = MigDevice.all()
        for gpu in mig:
            point = Point("gpu_loading").tag("index", gpu.index).field("GPU", gpu.memory_used())
            points.append(point)
        write_api.write(bucket=BUCKET, record=points)
        print("Write points at {0}".format(datetime.datetime.fromtimestamp(time.time()).strftime('%c')))
        time.sleep(SLEEP)
except KeyboardInterrupt:
    pass



In [ ]:
from influxdb_client import  Point, InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
import docker
import time
import datetime

SLEEP = 5
BUCKET = "docker"
ORGANIZATION = "my-org"
TOKEN = "BwsRD6-_mbeV8IQUJjcAKJuj2pZir0pp9Cy1bezT3z0MJ5fkqD5wmhY_l5cnCbYLlgxsy0L8GkuJ0PM_n-sk6Q=="
URL = "http://localhost:8076"



client = InfluxDBClient(url=URL, token=TOKEN, org=ORGANIZATION)
buckets_api = client.buckets_api()

if (buckets_api.find_bucket_by_name(BUCKET) == None):
    buckets_api.create_bucket(bucket_name=BUCKET, org=ORGANIZATION)

docker_client = docker.from_env()
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()
try:
    while True:
        points = []
        for index, container in enumerate(docker_client.containers.list(all=True)):
            if container.image.tags[0] in ["tf2:0.05", "tf1:0.01"]:
                if (container.status == "running"):
                    point = Point("docker_stat").tag("id", container.short_id).field("status", container.status).field("cpu", container.stats(stream=False)["cpu_stats"]["cpu_usage"]["total_usage"]).field("memory", container.stats(stream=False)["memory_stats"]["usage"])
                else:
                    point = Point("docker_stat").tag("id", container.short_id).field("status", container.status)
                points.append(point)
        write_api.write(bucket=BUCKET, record=points)
        print("Write docker state at {0}".format(datetime.datetime.fromtimestamp(time.time()).strftime('%c')))
        time.sleep(SLEEP)
except KeyboardInterrupt:
    pass

In [ ]:
for i in range(1000):
    _point1 = Point("my_measurement").tag("location", "Prague").field("temperature", 25.3* random.random())
    _point2 = Point("my_measurement").tag("location", "New York").field("temperature", 24.3* random.random())
    _point3 = Point("my_measurement").tag("location", "Moscow").field("temperature", 14.3* random.random())
    _point4 = Point("my_measurement").tag("location", "Tokyo").field("temperature", 28.3* random.random())
    _point5 = Point("my_measurement").tag("location", "Seoul").field("temperature", 25.3* random.random())
    write_api.write(bucket="my-bucket", record=[_point1, _point2, _point3, _point4, _point5])
    time.sleep(10)

In [37]:
tables = query_api.query('from(bucket:"my-bucket") |> range(start: -10m)')
for table in tables:
    print(table)
    for row in table.records:
        print (row.values)

FluxTable() columns: 9, records: 1
{'result': '_result', 'table': 0, '_start': datetime.datetime(2023, 3, 22, 5, 32, 57, 796437, tzinfo=datetime.timezone.utc), '_stop': datetime.datetime(2023, 3, 22, 5, 42, 57, 796437, tzinfo=datetime.timezone.utc), '_time': datetime.datetime(2023, 3, 22, 5, 42, 26, 229269, tzinfo=datetime.timezone.utc), '_value': 25.3, '_field': 'temperature', '_measurement': 'my_measurement', 'location': 'Prague'}
